# ...So
We now have a dictionary containing dataframes of FDA action dates and stock price time series. Lets open it up from dill and begin cutting out our feature space.

In [1]:
import dill
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

In [2]:
company_dataframes = dill.load(open('Prices_and_FDA_Dates.pkl', 'r'))
company_list = company_dataframes.keys()

In [3]:
len(company_list)

186

In [4]:
company_dataframes['NEW'].loc[company_dataframes['NEW']['pdufa?'] == True]

,volume,close,high,open,low,pdufa?
2006-10-06,0.0,11.6,12.8,12.8,11.6,True


In [5]:
testdf = company_dataframes['NEW']

In [6]:
testdf.reset_index(inplace = True)

In [7]:
testdf.loc[testdf['pdufa?']]

,index,volume,close,high,open,low,pdufa?
1699,2006-10-06,0.0,11.6,12.8,12.8,11.6,True


In [8]:
testdf.loc[1699]

index     2006-10-06
volume             0
close           11.6
high            12.8
open            12.8
low             11.6
pdufa?          True
Name: 1699, dtype: object

In [9]:
testdf.loc[1699]['close']

11.6

In [10]:
testind = testdf.index[testdf['pdufa?'] == True]

In [11]:
testind

Int64Index([1699], dtype='int64')

In [12]:
testind[0]

1699

So general idea:
1. iterate through dictionary of dataframes
1. in each dataframe, find each row with a `pdufa`
1. collect the (preceding and following) 120 rows of close prices and volumes
1. return those as an array of close price and volume vectors with the company name and pdufa date as metadata
1. something to the effect of `("Ticker", pdufaDate, (120 preceding close prices and volumes), (120 following close prices and volumes))`

In [13]:
data = []
for company in tqdm_notebook(company_list):
    df = company_dataframes[company].reset_index()
    pdufa_dates = df.index[df['pdufa?']].tolist()
    if len(pdufa_dates) > 0:
        for date in pdufa_dates:
            pRange = range(date-120, date)
            fRange = range(date, date+121)
            pCloses, pVolumes, fCloses, fVolumes = [], [], [], []
            for i in pRange:
                try:
                    pCloses.append(df.loc[i]['close'])
                    pVolumes.append(df.loc[i]['volume'])
                except:
                    pCloses.append(None)
                    pVolumes.append(None)
            for i in fRange:
                try:
                    fCloses.append(df.loc[i]['close'])
                    fVolumes.append(df.loc[i]['volume'])
                except:
                    fCloses.append(None)
                    fVolumes.append(None)
            data.append((company, df.loc[date]['index'], (pCloses, pVolumes), (fCloses, fVolumes)))

In [15]:
dill.dump(data, open('stock_price_training_slices.pkl', 'w'))

So theres our data points, stored as slices of the stock price/volume histories 120 days prior to an FDA event. `268*120*2 = 64320` data points in total. Time for some signal processing.

I know this could be done far more elegantly, but I need an adequate solution _yesterday_, not a perfect one next week.